In [32]:
from dotenv import load_dotenv
import os

In [28]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPEN_API_KEY')
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
#pinecone_env = os.getenv('PINECONE_ENV')

Embeddings

In [21]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

embedding = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=open_api_key)

In [24]:
#Embed the whole first Harry Potter Book

loader = TextLoader("harry_potter.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1071, which is longer than the specified 1000


Vector Stores with Pinecone

In [42]:
from langchain_pinecone import PineconeVectorStore

index_name = "ai-chatbot"
#docsearch = PineconeVectorStore.from_documents(docs, embedding, index_name=index_name)
docsearch = PineconeVectorStore(index_name=index_name, embedding=embedding)

Similarity Search With a Query

In [52]:
query = "What school is Harry Potter invited to attend?"

def getContext(query):
    #embedding_vector = embedding.embed_query(query)
    #docs = docsearch.similarity_search_by_vector(embedding_vector)
    docs = docsearch.similarity_search(query)
    return docs[0].page_content


In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [48]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Answer the question based on the following context:
{context}
---
Answer the question based on the above context: {query}
"""


prompt = ChatPromptTemplate.from_template(template)

In [51]:
messages = prompt.format(context=context, query=query)
result = llm.predict(messages)
print(result)

Harry Potter is invited to attend Hogwarts School of Witchcraft and Wizardry.


In [53]:
while(True):
    q = input("Ask anything about Harry Potter Sorcerer's Stone: ")
    context = getContext(q)
    messages = prompt.format(context=context, query=q)
    result = llm.predict(messages)
    print(result)